# Time Series with LSTMs

Cassandra Maldonado

1. Import, clean and plot the input data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
import tensorflow as tf
import os
from helper import timeseries_train_test_split

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

2. Train test split (use provided helper function helper.py Download helper.py, as we can’t use sklearn’s for time series data).

3. Fit a single layer LSTM with lag = 3 hours.

4. Fit a single layer LSTM with lag = 24 hours.

5. Fit a bidirectional LSTM with lag = 24 hours.

6. Plot results on test set and compare performance using RMSE.